# Сбор данных с видео-платформы

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Устанавливаем-и-загружаем-необходимые-библиотеки" data-toc-modified-id="Устанавливаем-и-загружаем-необходимые-библиотеки-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Устанавливаем и загружаем необходимые библиотеки</a></span></li><li><span><a href="#Применяем-API-ключ-и-channel_id-выбранного-канала" data-toc-modified-id="Применяем-API-ключ-и-channel_id-выбранного-канала-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Применяем API-ключ и channel_id выбранного канала</a></span></li><li><span><a href="#Создаём-датафрейм-с-общей-информацией-о-канале" data-toc-modified-id="Создаём-датафрейм-с-общей-информацией-о-канале-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Создаём датафрейм с общей информацией о канале</a></span></li><li><span><a href="#Создаём-функции-для-сбора-данных-о-видео" data-toc-modified-id="Создаём-функции-для-сбора-данных-о-видео-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Создаём функции для сбора данных о видео</a></span></li><li><span><a href="#Собираем-данные-о-видео,-создаём-датафрейм" data-toc-modified-id="Собираем-данные-о-видео,-создаём-датафрейм-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Собираем данные о видео, создаём датафрейм</a></span></li><li><span><a href="#Преобразуем-данные-о-длительности-видео" data-toc-modified-id="Преобразуем-данные-о-длительности-видео-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Преобразуем данные о длительности видео</a></span></li><li><span><a href="#Скачиваем-полученные-данные" data-toc-modified-id="Скачиваем-полученные-данные-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Скачиваем полученные данные</a></span></li></ul></div>

## Устанавливаем и загружаем необходимые библиотеки

При первом запуске нужно установить это:

In [1]:
#pip install --upgrade google-api-python-client

In [2]:
#pip install --upgrade google-auth-oauthlib google-auth-httplib2

In [3]:
import requests
import json
import pandas as pd
import numpy as np
import time
from datetime import datetime, date, timedelta
import googleapiclient.discovery
from IPython.display import FileLink, FileLinks

In [4]:
# убираем предупреждения
import warnings
warnings.filterwarnings("ignore")

## Применяем API-ключ и channel_id выбранного канала

API-ключ создаётся отдельно (оформляется в разделе для разработчиков), а channel_id узнаём на самой видео-платформе.
Добавляем эти данные в переменные:

In [5]:
API_KEY = '...' # нужно получить и вставить сюда свой ключ. Без него ничего не сработает

# для каждого канала свой id, узнаём и вставляем его сюда:
CHANNEL_ID='UCIFn8hAPOLr0m6A2qtnVVlg' # это например id канала Ванессы Мэй

In [6]:
api_service_name = "youtube"
api_version = "v3"
 
youtube = googleapiclient.discovery.build(
   api_service_name, api_version, developerKey = API_KEY)

## Создаём датафрейм с общей информацией о канале

In [7]:
r = youtube.channels().list(
        id = CHANNEL_ID,
        part = 'snippet, statistics').execute()

channel_main=pd.DataFrame(
    columns=['id','title', 'description','published_at','view_count','subscriber_count','video_count'])
channel_main=channel_main.append({'id':r['items'][0]['id'],
              'title':r['items'][0]['snippet']['title'],
              'description':r['items'][0]['snippet']['description'],
              'published_at':r['items'][0]['snippet']['publishedAt'], 
              'view_count':r['items'][0]['statistics']['viewCount'],
              'subscriber_count':r['items'][0]['statistics']['subscriberCount'],
              'video_count':r['items'][0]['statistics']['videoCount']}, ignore_index=True)

channel_main

,id,title,description,published_at,view_count,subscriber_count,video_count
0,UCIFn8hAPOLr0m6A2qtnVVlg,Vanessa-Mae,The official channel the violin player Vanessa...,2019-02-15T16:52:54Z,41964316,165000,10


## Создаём функции для сбора данных о видео 

In [8]:
 def get_video_details(video_id):
    # создаём запрос для получения данных о просмотрах, лайках, комментариях и продолжительности видео
    url_video_stats = ('https://www.googleapis.com/youtube/v3/videos?id='+video_id+
                       '&part=statistics&key='+API_KEY+'&part=contentDetails')
    response_video_stats = requests.get(url_video_stats).json()
        
    # сохраняем данные в переменные    
    view_count = response_video_stats['items'][0]['statistics']['viewCount']
    like_count = response_video_stats['items'][0]['statistics']['likeCount']
    comment_count = response_video_stats['items'][0]['statistics']['commentCount']
    duration = response_video_stats['items'][0]['contentDetails']['duration']
        
    return view_count, like_count, comment_count, duration

In [9]:
def get_videos(df):
    # делаем запрос для получения списка видео на канале, сортируем по дате выхода видео
    # максимально возможное количество видео для одного запроса - 50
    pageToken = ''
    while 1:
        url = ('https://www.googleapis.com/youtube/v3/search?key='+API_KEY+
               '&channelId='+CHANNEL_ID+'&part=snippet,id&order=date&maxResults=50&'+pageToken)
        response = requests.get(url).json()
        time.sleep(1) 
        
        # обрабатываем общие данные о видео
        for video in response['items']:
            if video['id']['kind'] == "youtube#video":
                video_id = video['id']['videoId']
                video_title = video['snippet']['title']
                video_title = str(video_title).replace('&amp;','')
                video_title = str(video_title).replace('@',' ')
                video_title = str(video_title).replace('&#39;','')
                video_title = str(video_title).replace('&quot;','`')
                upload_date = video['snippet']['publishedAt']
                upload_date = str(upload_date).split("T")[0]

                # детальную информацию о каждом видео получаем при помощи ранее созданной функции
                view_count, like_count, comment_count, duration = get_video_details(video_id)
                
                # сохраняем данные в датафрейм
                df = df.append({'video_id':video_id,
                                'video_title':video_title,
                                'upload_date':upload_date,
                                'duration':duration,
                                'view_count':view_count,
                                'like_count':like_count,
                                'comment_count':comment_count},
                               ignore_index=True)
        try:
            if response['nextPageToken'] != None: 
                pageToken = 'pageToken=' + response['nextPageToken']

        except:
            break


    return df

## Собираем данные о видео, создаём датафрейм

In [10]:
df = pd.DataFrame(columns=['video_id','video_title','upload_date', 'duration','view_count','like_count','comment_count'])
df = get_videos(df)

df = df.sort_values(by='upload_date').reset_index(drop=True)
df

,video_id,video_title,upload_date,duration,view_count,like_count,comment_count
0,g-p7ewn7mB0,Vanessa-Mae - Red Hot (Official Video),2019-05-09,PT3M56S,1062716,8659,269
1,T3s2dn5X6Xk,Vanessa-Mae - Im A Doun For Lack O Johnnie (Of...,2019-05-09,PT4M39S,333873,3296,115
2,USACuJCrZPY,Vanessa-Mae - The Devils Trill Sonata (Officia...,2019-05-09,PT3M39S,5077441,43052,1088
3,VuqcO8uy1ek,Vanessa-Mae - White Bird [Airscape Edit] (Offi...,2019-05-09,PT3M51S,19210,351,12
4,fbkcAkBuZK8,Vanessa-Mae - Classical Gas (Official Video),2019-05-09,PT3M52S,216694,2354,85
5,mdFrn89x74k,Vanessa-Mae - Storm (Official Video),2019-05-09,PT3M57S,23011626,193330,4565
6,8M1_jF5nmZQ,Vanessa-Mae - White Bird (Official Video),2019-05-09,PT3M53S,207240,2001,114
7,mA-21LW_gCM,Vanessa-Mae - I Feel Love (Official Video),2019-05-09,PT4M9S,201151,2903,155
8,Xh5eCupjS1o,Vanessa-Mae - Toccata and Fugue in D Minor (Of...,2019-05-09,PT3M51S,2279857,23425,641
9,9Iui4K6H7Rc,Vanessa-Mae - Destiny (Official Video),2019-05-09,PT3M6S,1459323,13526,352


## Преобразуем данные о длительности видео

In [11]:
# выясняем, где есть данные о часах, минутах, секундах
df['index_H'] = df['duration'].str.find('H')
df['index_M'] = df['duration'].str.find('M')
df['index_S'] = df['duration'].str.find('S')

# в зависимости от набора вариантов делим датасет на части по условиям
df_HMS = df.query('index_H != -1 and index_M != -1 and index_S != -1')
df_HM = df.query('index_H != -1 and index_M != -1 and index_S == -1')
df_HS = df.query('index_H != -1 and index_M == -1 and index_S != -1')
df_H = df.query('index_H != -1 and index_M == -1 and index_S == -1')
df_MS = df.query('index_H == -1 and index_M != -1 and index_S != -1')
df_M = df.query('index_H == -1 and index_M != -1 and index_S == -1')
df_S = df.query('index_H == -1 and index_M == -1 and index_S != -1')

In [12]:
# для каждого куска датасета создаём данные о часах, минутах, секундах
# так код сработает для любого видео: хоть в нём есть часы, минуты, секунды; или только часы и минуты; 
# или только часы и секунды и т.д.
if len(df_HMS) > 0:
    df_HMS['duration_hours'] = df_HMS['duration'].str.split('H', 1, expand=True)[0].str.strip('PT')
    df_HMS['duration_minutes'] = df_HMS['duration'].str.split('H', 1, expand=True)[1].str.split('M', 1, expand=True)[0]
    df_HMS['duration_seconds']= (df_HMS['duration'].str.split('H',1,expand=True)[1].str.split('M',1,expand=True)[1]
                                 .str.strip('S'))

if len(df_HM) > 0:
    df_HM['duration_hours'] = df_HM['duration'].str.split('H', 1, expand=True)[0].str.strip('PT')
    df_HM['duration_minutes'] = df_HM['duration'].str.split('H', 1, expand=True)[1].str.strip('M')
    df_HM['duration_seconds'] = 0

if len(df_HS) > 0:
    df_HS['duration_hours'] = df_HS['duration'].str.split('H', 1, expand=True)[0].str.strip('PT')
    df_HS['duration_minutes'] = 0
    df_HS['duration_seconds'] = df_HS['duration'].str.split('H', 1, expand=True)[1].str.strip('S')

if len(df_H) > 0:
    df_H['duration_hours'] = df_H['duration'].str.split('H', 1, expand=True)[0].str.strip('PT')
    df_H['duration_minutes'] = 0
    df_H['duration_seconds'] = 0

if len(df_MS) > 0:
    df_MS['duration_hours'] = 0
    df_MS['duration_minutes'] = df_MS['duration'].str.split('M', 1, expand=True)[0].str.strip('PT')
    df_MS['duration_seconds'] = df_MS['duration'].str.split('M', 1, expand=True)[1].str.strip('S')

if len(df_M) > 0:
    df_M['duration_hours'] = 0
    df_M['duration_minutes'] = df_M['duration'].str.split('M', 1, expand=True)[0].str.strip('PT')
    df_M['duration_seconds'] = 0

if len(df_S) > 0:
    df_S['duration_hours'] = 0
    df_S['duration_minutes'] = 0
    df_S['duration_seconds'] = df_S['duration'].str.split('S', 1, expand=True)[0].str.strip('PT')

# соединяем датасеты назад в один
df = pd.concat([df_HMS, df_HM, df_HS, df_H, df_MS, df_M, df_S])

# переводим столбцы в нужный формат
df[['duration_hours', 'duration_minutes', 'duration_seconds']] = (
    df[['duration_hours', 'duration_minutes', 'duration_seconds']].astype('int'))

# создаём столбец с суммарной длительностью в секундах
df['duration_seconds_total'] = (df['duration_hours']*60*60 + df['duration_minutes']*60 + df['duration_seconds'])

# результат: длительность видео в привычном формате
df['video_duration'] = df['duration_seconds_total'].apply(lambda x: str(timedelta(seconds =x)))

# для финального датасета убираем ненужные столбцы
df = df.drop(columns=['index_H', 'index_M', 'index_S', 'duration_hours', 'duration_minutes', 'duration_seconds'])

# можно сверить данные в столбцах duration и video_duration
df

,video_id,video_title,upload_date,duration,view_count,like_count,comment_count,duration_seconds_total,video_duration
0,g-p7ewn7mB0,Vanessa-Mae - Red Hot (Official Video),2019-05-09,PT3M56S,1062716,8659,269,236,0:03:56
1,T3s2dn5X6Xk,Vanessa-Mae - Im A Doun For Lack O Johnnie (Of...,2019-05-09,PT4M39S,333873,3296,115,279,0:04:39
2,USACuJCrZPY,Vanessa-Mae - The Devils Trill Sonata (Officia...,2019-05-09,PT3M39S,5077441,43052,1088,219,0:03:39
3,VuqcO8uy1ek,Vanessa-Mae - White Bird [Airscape Edit] (Offi...,2019-05-09,PT3M51S,19210,351,12,231,0:03:51
4,fbkcAkBuZK8,Vanessa-Mae - Classical Gas (Official Video),2019-05-09,PT3M52S,216694,2354,85,232,0:03:52
5,mdFrn89x74k,Vanessa-Mae - Storm (Official Video),2019-05-09,PT3M57S,23011626,193330,4565,237,0:03:57
6,8M1_jF5nmZQ,Vanessa-Mae - White Bird (Official Video),2019-05-09,PT3M53S,207240,2001,114,233,0:03:53
7,mA-21LW_gCM,Vanessa-Mae - I Feel Love (Official Video),2019-05-09,PT4M9S,201151,2903,155,249,0:04:09
8,Xh5eCupjS1o,Vanessa-Mae - Toccata and Fugue in D Minor (Of...,2019-05-09,PT3M51S,2279857,23425,641,231,0:03:51
9,9Iui4K6H7Rc,Vanessa-Mae - Destiny (Official Video),2019-05-09,PT3M6S,1459323,13526,352,186,0:03:06


In [13]:
# теперь убираем столбец duration с исходными данными в формате ISO 8601
df = df.drop(columns=['duration'])

# переименовываем столбцы с результатами наших преобразований в более лаконичные
df.rename(columns={'video_duration':'duration', 'duration_seconds_total':'duration_seconds'}, inplace=True) 

df

,video_id,video_title,upload_date,view_count,like_count,comment_count,duration_seconds,duration
0,g-p7ewn7mB0,Vanessa-Mae - Red Hot (Official Video),2019-05-09,1062716,8659,269,236,0:03:56
1,T3s2dn5X6Xk,Vanessa-Mae - Im A Doun For Lack O Johnnie (Of...,2019-05-09,333873,3296,115,279,0:04:39
2,USACuJCrZPY,Vanessa-Mae - The Devils Trill Sonata (Officia...,2019-05-09,5077441,43052,1088,219,0:03:39
3,VuqcO8uy1ek,Vanessa-Mae - White Bird [Airscape Edit] (Offi...,2019-05-09,19210,351,12,231,0:03:51
4,fbkcAkBuZK8,Vanessa-Mae - Classical Gas (Official Video),2019-05-09,216694,2354,85,232,0:03:52
5,mdFrn89x74k,Vanessa-Mae - Storm (Official Video),2019-05-09,23011626,193330,4565,237,0:03:57
6,8M1_jF5nmZQ,Vanessa-Mae - White Bird (Official Video),2019-05-09,207240,2001,114,233,0:03:53
7,mA-21LW_gCM,Vanessa-Mae - I Feel Love (Official Video),2019-05-09,201151,2903,155,249,0:04:09
8,Xh5eCupjS1o,Vanessa-Mae - Toccata and Fugue in D Minor (Of...,2019-05-09,2279857,23425,641,231,0:03:51
9,9Iui4K6H7Rc,Vanessa-Mae - Destiny (Official Video),2019-05-09,1459323,13526,352,186,0:03:06


## Скачиваем полученные данные

In [14]:
channel_main.to_excel('/datasets/video_collection/vanessa_mae_main.xlsx', index=False)

df.to_excel('/datasets/video_collection/vanessa_mae_table.xlsx', index=False)